# Gemma 因果推理选择题测试

本notebook用于测试Gemma模型在因果推理选择题上的表现：
1. 从已回答的数据中获取问题index
2. 根据index从原始数据集中获取完整题目（premise, hypothesis1, hypothesis2）
3. 让Gemma直接选择答案
4. 计算总正确率及按ask-for类型分类的正确率

In [1]:
# Cell 1: 导入库和配置
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os
from tqdm import tqdm
from collections import defaultdict

# =================配置区域=================
# 模型路径配置
CACHE_DIR = "/home/amax/Gemma_and_NVembed/Gemma"
MODEL_NAME = "google/gemma-2-9b-it"
# 数据路径配置
ANSWERED_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/T_1.25_Gemma_Answer/gemma_causal_answers_100.jsonl"
TRAIN_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Ecare/train.jsonl"
OUTPUT_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl"

print("配置加载完成！")

/home/amax/miniconda3/envs/gemma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


配置加载完成！


In [2]:
# Cell 2: 加载模型
def setup_model():
    """加载本地 Gemma 模型"""
    print(f"正在加载模型: {MODEL_NAME} ...")
    
    # 查找本地模型路径
    model_base = os.path.join(CACHE_DIR, "models--google--gemma-2-9b-it", "snapshots")
    
    if not os.path.exists(model_base):
        raise FileNotFoundError(f"模型目录不存在: {model_base}")
    
    snapshots = os.listdir(model_base)
    if not snapshots:
        raise FileNotFoundError("模型尚未下载，请先下载模型")
    
    local_model_path = os.path.join(model_base, snapshots[0])
    print(f"📁 本地模型路径: {local_model_path}")
    
    # 加载 tokenizer
    print("\n⏳ 正在加载 tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        local_model_path,
        local_files_only=True
    )
    print("✅ Tokenizer 加载成功！")
    
    # 加载模型
    print("\n⏳ 正在加载模型（约需 1-2 分钟）...")
    model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        local_files_only=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    print(f"✅ 模型加载成功！")
    print(f"💻 模型设备: {model.device}")
    print(f"📊 模型参数量: {model.num_parameters() / 1e9:.2f}B")
    
    return tokenizer, model

# 加载模型
tokenizer, model = setup_model()

正在加载模型: google/gemma-2-9b-it ...
📁 本地模型路径: /home/amax/Gemma_and_NVembed/Gemma/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819

⏳ 正在加载 tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Tokenizer 加载成功！

⏳ 正在加载模型（约需 1-2 分钟）...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.39it/s]

✅ 模型加载成功！
💻 模型设备: cuda:0
📊 模型参数量: 9.24B


In [3]:
# Cell 3: 加载数据并构建问题集
def load_answered_indices(answered_file):
    """从已回答文件中提取unique的index"""
    indices = set()
    with open(answered_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                record = json.loads(line)
                indices.add(record['index'])
    return indices

def load_train_data(train_file):
    """加载训练数据，返回以index为key的字典"""
    data_dict = {}
    with open(train_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                record = json.loads(line)
                data_dict[record['index']] = record
    return data_dict

def prepare_questions(answered_indices, train_data):
    """根据已回答的indices准备问题数据"""
    questions = []
    for idx in answered_indices:
        if idx in train_data:
            questions.append(train_data[idx])
        else:
            print(f"⚠️ Index {idx} 在训练集中未找到")
    return questions

# 加载数据
print("📂 加载已回答的问题indices...")
answered_indices = load_answered_indices(ANSWERED_FILE)
print(f"📊 已回答的unique index数: {len(answered_indices)}")

print("\n📂 加载训练数据集...")
train_data = load_train_data(TRAIN_FILE)
print(f"📊 训练集总样本数: {len(train_data)}")

print("\n📂 准备问题数据...")
questions = prepare_questions(answered_indices, train_data)
print(f"📊 待测试问题数: {len(questions)}")

# 统计ask-for类型分布
effect_count = sum(1 for q in questions if q['ask-for'] == 'effect')
cause_count = sum(1 for q in questions if q['ask-for'] == 'cause')
print(f"\n📈 问题类型分布:")
print(f"   - Effect类型: {effect_count}")
print(f"   - Cause类型: {cause_count}")

📂 加载已回答的问题indices...
📊 已回答的unique index数: 795

📂 加载训练数据集...
📊 训练集总样本数: 14928

📂 准备问题数据...
📊 待测试问题数: 795

📈 问题类型分布:
   - Effect类型: 359
   - Cause类型: 436


In [4]:
# Cell 4: 构造prompt并让Gemma回答选择题
def construct_choice_prompt(question):
    """
    构造选择题prompt
    
    Args:
        question: 包含 premise, ask-for, hypothesis1, hypothesis2 的字典
    
    Returns:
        prompt字符串
    """
    premise = question['premise']
    ask_for = question['ask-for']
    h1 = question['hypothesis1']
    h2 = question['hypothesis2']
    
    if ask_for == "effect":
        prompt = f"""You are a causal reasoning expert. Given a premise, select the most likely effect from the two options.

Premise: {premise}

Options:
A. {h1}
B. {h2}

Please select the correct answer (A or B only). Just output the letter, nothing else.

Answer:"""
    else:  # cause
        prompt = f"""You are a causal reasoning expert. Given a result, select the most likely cause from the two options.

Result: {premise}

Options:
A. {h1}
B. {h2}

Please select the correct answer (A or B only). Just output the letter, nothing else.

Answer:"""
    
    return prompt


def get_gemma_choice(tokenizer, model, prompt):
    """
    让Gemma回答选择题
    
    Args:
        tokenizer: 分词器
        model: 语言模型
        prompt: 输入prompt
    
    Returns:
        'A', 'B' 或 'unknown'
    """
    # 使用 chat template
    chat = [{"role": "user", "content": prompt}]
    prompt_formatted = tokenizer.apply_chat_template(
        chat, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt_formatted, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,  # 只需要生成很短的回答
            do_sample=False,    # 使用greedy decoding确保一致性
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 只提取新生成的部分
    generated_text = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:], 
        skip_special_tokens=True
    ).strip().upper()
    
    # 解析答案
    if 'A' in generated_text[:5]:
        return 'A', generated_text
    elif 'B' in generated_text[:5]:
        return 'B', generated_text
    else:
        return 'unknown', generated_text


def run_choice_test(tokenizer, model, questions, output_file):
    """
    运行选择题测试
    
    Args:
        tokenizer: 分词器
        model: 语言模型
        questions: 问题列表
        output_file: 输出文件路径
    
    Returns:
        results: 结果列表
    """
    results = []
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    print(f"\n🚀 开始选择题测试...")
    print(f"📝 输出文件: {output_file}")
    print(f"📊 总问题数: {len(questions)}")
    
    with open(output_file, 'w', encoding='utf-8') as f:
        for question in tqdm(questions, desc="回答选择题"):
            # 构造prompt
            prompt = construct_choice_prompt(question)
            
            # 获取Gemma的选择
            choice, raw_answer = get_gemma_choice(tokenizer, model, prompt)
            
            # 判断正确答案（label=0表示hypothesis1正确，即A正确）
            correct_answer = 'A' if question['label'] == 0 else 'B'
            is_correct = (choice == correct_answer)
            
            # 构建结果记录
            record = {
                "index": question['index'],
                "premise": question['premise'],
                "ask-for": question['ask-for'],
                "hypothesis1": question['hypothesis1'],
                "hypothesis2": question['hypothesis2'],
                "label": question['label'],
                "correct_answer": correct_answer,
                "gemma_choice": choice,
                "gemma_raw_answer": raw_answer,
                "is_correct": is_correct
            }
            
            results.append(record)
            
            # 写入文件
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
    
    print(f"\n✅ 测试完成！结果已保存到: {output_file}")
    return results

# 运行测试
results = run_choice_test(tokenizer, model, questions, OUTPUT_FILE)


🚀 开始选择题测试...
📝 输出文件: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl
📊 总问题数: 795


回答选择题:   0%|          | 0/795 [00:00<?, ?it/s]

回答选择题: 100%|██████████| 795/795 [01:17<00:00, 10.20it/s]


✅ 测试完成！结果已保存到: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl


In [5]:
# Cell 5: 计算正确率
def calculate_accuracy(results):
    """
    计算总正确率和按ask-for分类的正确率
    
    Args:
        results: 测试结果列表
    
    Returns:
        统计字典
    """
    # 总体统计
    total = len(results)
    correct = sum(1 for r in results if r['is_correct'])
    unknown = sum(1 for r in results if r['gemma_choice'] == 'unknown')
    
    # 按ask-for分类统计
    effect_results = [r for r in results if r['ask-for'] == 'effect']
    cause_results = [r for r in results if r['ask-for'] == 'cause']
    
    effect_total = len(effect_results)
    effect_correct = sum(1 for r in effect_results if r['is_correct'])
    effect_unknown = sum(1 for r in effect_results if r['gemma_choice'] == 'unknown')
    
    cause_total = len(cause_results)
    cause_correct = sum(1 for r in cause_results if r['is_correct'])
    cause_unknown = sum(1 for r in cause_results if r['gemma_choice'] == 'unknown')
    
    stats = {
        'total': {
            'total': total,
            'correct': correct,
            'unknown': unknown,
            'accuracy': correct / total * 100 if total > 0 else 0,
            'valid_accuracy': correct / (total - unknown) * 100 if (total - unknown) > 0 else 0
        },
        'effect': {
            'total': effect_total,
            'correct': effect_correct,
            'unknown': effect_unknown,
            'accuracy': effect_correct / effect_total * 100 if effect_total > 0 else 0,
            'valid_accuracy': effect_correct / (effect_total - effect_unknown) * 100 if (effect_total - effect_unknown) > 0 else 0
        },
        'cause': {
            'total': cause_total,
            'correct': cause_correct,
            'unknown': cause_unknown,
            'accuracy': cause_correct / cause_total * 100 if cause_total > 0 else 0,
            'valid_accuracy': cause_correct / (cause_total - cause_unknown) * 100 if (cause_total - cause_unknown) > 0 else 0
        }
    }
    
    return stats


def print_statistics(stats):
    """打印统计结果"""
    print("\n" + "=" * 60)
    print("📊 Gemma 因果推理选择题测试结果")
    print("=" * 60)
    
    print("\n📈 总体正确率:")
    print(f"   总问题数: {stats['total']['total']}")
    print(f"   正确回答: {stats['total']['correct']}")
    print(f"   无效回答: {stats['total']['unknown']}")
    print(f"   总正确率: {stats['total']['accuracy']:.2f}%")
    print(f"   有效正确率 (排除unknown): {stats['total']['valid_accuracy']:.2f}%")
    
    print("\n📈 Effect类型正确率:")
    print(f"   总问题数: {stats['effect']['total']}")
    print(f"   正确回答: {stats['effect']['correct']}")
    print(f"   无效回答: {stats['effect']['unknown']}")
    print(f"   正确率: {stats['effect']['accuracy']:.2f}%")
    print(f"   有效正确率: {stats['effect']['valid_accuracy']:.2f}%")
    
    print("\n📈 Cause类型正确率:")
    print(f"   总问题数: {stats['cause']['total']}")
    print(f"   正确回答: {stats['cause']['correct']}")
    print(f"   无效回答: {stats['cause']['unknown']}")
    print(f"   正确率: {stats['cause']['accuracy']:.2f}%")
    print(f"   有效正确率: {stats['cause']['valid_accuracy']:.2f}%")
    
    print("\n" + "=" * 60)


# 计算并打印统计结果
stats = calculate_accuracy(results)
print_statistics(stats)

# 保存统计结果
stats_file = OUTPUT_FILE.replace('.jsonl', '_stats.json')
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)
print(f"\n📁 统计结果已保存到: {stats_file}")


📊 Gemma 因果推理选择题测试结果

📈 总体正确率:
   总问题数: 795
   正确回答: 667
   无效回答: 1
   总正确率: 83.90%
   有效正确率 (排除unknown): 84.01%

📈 Effect类型正确率:
   总问题数: 359
   正确回答: 299
   无效回答: 0
   正确率: 83.29%
   有效正确率: 83.29%

📈 Cause类型正确率:
   总问题数: 436
   正确回答: 368
   无效回答: 1
   正确率: 84.40%
   有效正确率: 84.60%


📁 统计结果已保存到: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results_stats.json


In [6]:
# Cell 6: 查看示例结果
def show_sample_results(results, n=5):
    """显示部分结果示例"""
    print("\n📋 正确回答示例:")
    print("-" * 60)
    correct_examples = [r for r in results if r['is_correct']][:n]
    for i, r in enumerate(correct_examples, 1):
        print(f"\n[{i}] Index: {r['index']}")
        print(f"    Premise: {r['premise']}")
        print(f"    Ask-for: {r['ask-for']}")
        print(f"    A: {r['hypothesis1']}")
        print(f"    B: {r['hypothesis2']}")
        print(f"    正确答案: {r['correct_answer']} | Gemma选择: {r['gemma_choice']} ✅")
    
    print("\n\n📋 错误回答示例:")
    print("-" * 60)
    wrong_examples = [r for r in results if not r['is_correct'] and r['gemma_choice'] != 'unknown'][:n]
    for i, r in enumerate(wrong_examples, 1):
        print(f"\n[{i}] Index: {r['index']}")
        print(f"    Premise: {r['premise']}")
        print(f"    Ask-for: {r['ask-for']}")
        print(f"    A: {r['hypothesis1']}")
        print(f"    B: {r['hypothesis2']}")
        print(f"    正确答案: {r['correct_answer']} | Gemma选择: {r['gemma_choice']} ❌")
        print(f"    Gemma原始回答: {r['gemma_raw_answer']}")
    
    # 显示无效回答
    unknown_examples = [r for r in results if r['gemma_choice'] == 'unknown'][:3]
    if unknown_examples:
        print("\n\n📋 无效回答示例:")
        print("-" * 60)
        for i, r in enumerate(unknown_examples, 1):
            print(f"\n[{i}] Index: {r['index']}")
            print(f"    Gemma原始回答: {r['gemma_raw_answer']}")

# 显示示例结果
show_sample_results(results)


📋 正确回答示例:
------------------------------------------------------------

[1] Index: train-12680
    Premise: Tom didn't hear what the boss has said.
    Ask-for: effect
    A: Tom disobeyed the new principle.
    B: Tom couldn't be able to know the taste of whatever he eats.
    正确答案: A | Gemma选择: A ✅

[2] Index: train-7575
    Premise: He saw fins.
    Ask-for: cause
    A: Little Jack stared at the inner part of the guppy.
    B: Little Jack stared at the anal part of the guppy.
    正确答案: B | Gemma选择: B ✅

[3] Index: train-4374
    Premise: The child is affected by herpes virus.
    Ask-for: effect
    A: The child must receive immediate treatment.
    B: Children infected with the virus will experience gastrointestinal discomfort.
    正确答案: A | Gemma选择: A ✅

[4] Index: train-1824
    Premise: Mendelevium is a unique element.
    Ask-for: cause
    A: Tom has kept carbides under a low temperature.
    B: Lack of  naturally occurring isotopes means uniqueness.
    正确答案: B | Gemma选择: B

# Part 2: 错误题目的聚类分析与再次验证 (独立运行)

**此部分可独立运行，无需先执行 Part 1**

对于 Gemma 直接选择错误的题目：
1. 从文件读取 Gemma 选择结果 (gemma_choice_results.jsonl)
2. 加载 NV-Embed 模型，对其 100 次回答进行向量化
3. 使用 HDBSCAN 聚类，找到最大 cluster
4. 计算 core point 和 topk 代表向量对应的原始回答
5. 加载 Gemma 模型，判断这些代表性回答与哪个选项 (H1/H2) 更接近

In [1]:
# Cell 7: Part2 独立运行 - 导入库、配置和加载 NV-Embed 模型
# ========== 导入库 ==========
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import numpy as np
import json
import os
from tqdm import tqdm
from collections import defaultdict
import hdbscan
from scipy.spatial.distance import cdist

# ========== 配置区域 ==========
# NV-Embed 模型路径
NVEMBED_PATH = "/home/amax/Gemma_and_NVembed/NV/snapshots/main"

# 数据路径配置
GEMMA_CHOICE_RESULTS_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl"
ANSWERED_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/T_1.25_Gemma_Answer/gemma_causal_answers_100.jsonl"
OUTPUT_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl"

print("配置加载完成！")

# 加载 NV-Embed 模型
print("=" * 60)
print("加载 NV-Embed 模型...")
print("=" * 60)

nvembed_tokenizer = AutoTokenizer.from_pretrained(
    NVEMBED_PATH,
    trust_remote_code=True,
    local_files_only=True
)

nvembed_model = AutoModel.from_pretrained(
    NVEMBED_PATH,
    trust_remote_code=True,
    local_files_only=True,
    torch_dtype=torch.float16
)

device = "cuda" if torch.cuda.is_available() else "cpu"
nvembed_model = nvembed_model.to(device)
nvembed_model.eval()

print(f"✅ NV-Embed 模型加载成功！设备: {device}")


def encode_texts_nvembed(texts, max_length=4096, batch_size=32):
    """
    使用 NV-Embed 批量编码文本为向量
    """
    if isinstance(texts, str):
        texts = [texts]
    
    eos_token = nvembed_tokenizer.eos_token if nvembed_tokenizer.eos_token else "</s>"
    texts_with_eos = [text + eos_token for text in texts]
    
    all_embeddings = []
    
    for i in range(0, len(texts_with_eos), batch_size):
        batch_texts = texts_with_eos[i:i+batch_size]
        
        batch_dict = nvembed_tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        batch_dict = {k: v.to(device) for k, v in batch_dict.items()}
        
        attention_mask = batch_dict["attention_mask"]
        seq_lengths = attention_mask.sum(dim=1)
        pool_mask = torch.zeros_like(attention_mask)
        for j, length in enumerate(seq_lengths):
            pool_mask[j, length - 1] = 1
        
        with torch.no_grad():
            outputs = nvembed_model(
                input_ids=batch_dict["input_ids"],
                attention_mask=attention_mask,
                pool_mask=pool_mask
            )
            
            if isinstance(outputs, dict):
                if "sentence_embeddings" in outputs:
                    embeddings = outputs["sentence_embeddings"]
                elif "last_hidden_state" in outputs:
                    embeddings = outputs["last_hidden_state"][:, -1, :]
                else:
                    raise ValueError(f"无法从输出中提取嵌入向量: {outputs.keys()}")
            elif hasattr(outputs, "sentence_embeddings"):
                embeddings = outputs.sentence_embeddings
            elif hasattr(outputs, "last_hidden_state"):
                embeddings = outputs.last_hidden_state[:, -1, :]
            else:
                embeddings = outputs
        
        embeddings = F.normalize(embeddings, p=2, dim=1)
        embeddings_cpu = embeddings.float().cpu()
        all_embeddings.append(np.array(embeddings_cpu.tolist()))
    
    return np.vstack(all_embeddings)

print("✅ encode_texts_nvembed 函数定义完成！")

/home/amax/miniconda3/envs/nvembed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


配置加载完成！
加载 NV-Embed 模型...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.12it/s]


✅ NV-Embed 模型加载成功！设备: cuda
✅ encode_texts_nvembed 函数定义完成！


In [2]:
# Cell 8: 从文件读取 Gemma 选择结果，找出错误题目并加载其100次回答
print("=" * 60)
print("从文件读取 Gemma 选择结果...")
print("=" * 60)

# ========== 从文件读取 Gemma 选择结果 ==========
results = []
with open(GEMMA_CHOICE_RESULTS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            results.append(json.loads(line))

print(f"📊 从文件读取了 {len(results)} 条选择结果")

# 统计正确率
total_correct = sum(1 for r in results if r['is_correct'])
print(f"📊 原始正确率: {total_correct}/{len(results)} = {total_correct/len(results)*100:.2f}%")

print("\n" + "=" * 60)
print("找出 Gemma 回答错误的题目...")
print("=" * 60)

# 获取错误题目的 index
wrong_results = [r for r in results if not r['is_correct'] and r['gemma_choice'] != 'unknown']
wrong_indices = [r['index'] for r in wrong_results]

print(f"📊 错误题目数量: {len(wrong_indices)}")

# 加载 Gemma 的 100 次回答
print("\n📂 加载 Gemma 的 100 次回答...")
gemma_100_answers = defaultdict(list)

with open(ANSWERED_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            record = json.loads(line)
            idx = record['index']
            if idx in wrong_indices:
                gemma_100_answers[idx].append({
                    'answer': record['Gemma_answer'],
                    'round': record['answer_round'],
                    'ask_for': record['ask-for'],
                    'premise': record['premise']
                })

print(f"✅ 加载完成！")
print(f"📊 有 100 次回答的错误题目数: {len([k for k, v in gemma_100_answers.items() if len(v) >= 100])}")

# 创建 index -> wrong_result 的映射
wrong_results_dict = {r['index']: r for r in wrong_results}

从文件读取 Gemma 选择结果...
📊 从文件读取了 800 条选择结果
📊 原始正确率: 672/800 = 84.00%

找出 Gemma 回答错误的题目...
📊 错误题目数量: 127

📂 加载 Gemma 的 100 次回答...
✅ 加载完成！
📊 有 100 次回答的错误题目数: 127


In [3]:
# Cell 9: 对错误题目进行 HDBSCAN 聚类，找到 max cluster 的 core_point 和 topk
print("=" * 60)
print("对错误题目进行 HDBSCAN 聚类分析...")
print("=" * 60)

# HDBSCAN 配置
HDBSCAN_MIN_CLUSTER_SIZE = 5
HDBSCAN_MIN_SAMPLES = 3

def analyze_wrong_question(idx, answers_list, wrong_info):
    """
    分析单个错误题目：聚类并找到 max cluster 的代表向量
    
    Returns:
        dict: 包含 core_point_text, topk_text 等信息
    """
    # 获取所有回答文本
    answer_texts = [a['answer'] for a in answers_list[:100]]
    
    # 向量化所有回答
    answer_embeddings = encode_texts_nvembed(answer_texts)
    
    # HDBSCAN 聚类
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
        min_samples=HDBSCAN_MIN_SAMPLES,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    cluster_labels = clusterer.fit_predict(answer_embeddings)
    
    # 找到各个 cluster 的大小
    unique_clusters = set(cluster_labels)
    cluster_sizes = {}
    for cid in unique_clusters:
        if cid >= 0:  # 排除噪声点 (-1)
            cluster_sizes[cid] = np.sum(cluster_labels == cid)
    
    if not cluster_sizes:
        # 没有有效聚类，使用全部数据
        max_cluster_id = -1
        cluster_mask = np.ones(len(answer_texts), dtype=bool)
    else:
        # 找到最大的 cluster
        max_cluster_id = max(cluster_sizes, key=cluster_sizes.get)
        cluster_mask = cluster_labels == max_cluster_id
    
    cluster_indices = np.where(cluster_mask)[0]
    cluster_embeddings = answer_embeddings[cluster_mask]
    cluster_texts = [answer_texts[i] for i in cluster_indices]
    
    # 计算 centroid
    centroid = np.mean(cluster_embeddings, axis=0)
    centroid = centroid / (np.linalg.norm(centroid) + 1e-12)
    
    # 计算 core_point (到其他点平均距离最小的点)
    if len(cluster_embeddings) > 1:
        pairwise_distances = cdist(cluster_embeddings, cluster_embeddings, metric='cosine')
        avg_distances = np.mean(pairwise_distances, axis=1)
        core_idx_in_cluster = np.argmin(avg_distances)
        core_point_vec = cluster_embeddings[core_idx_in_cluster]
        core_point_text = cluster_texts[core_idx_in_cluster]
    else:
        core_idx_in_cluster = 0
        core_point_vec = cluster_embeddings[0]
        core_point_text = cluster_texts[0]
    
    # 计算 topk (距离 centroid 最近的 k 个点的平均)
    distances_to_centroid = cdist([centroid], cluster_embeddings, metric='cosine')[0]
    k = min(3, len(cluster_embeddings))
    topk_indices_in_cluster = np.argsort(distances_to_centroid)[:k]
    topk_vec = np.mean(cluster_embeddings[topk_indices_in_cluster], axis=0)
    topk_vec = topk_vec / (np.linalg.norm(topk_vec) + 1e-12)
    topk_texts = [cluster_texts[i] for i in topk_indices_in_cluster]
    
    # 找到距离 topk_vec 最近的原始文本
    topk_distances = cdist([topk_vec], cluster_embeddings, metric='cosine')[0]
    closest_to_topk_idx = np.argmin(topk_distances)
    topk_representative_text = cluster_texts[closest_to_topk_idx]
    
    return {
        'index': idx,
        'premise': wrong_info['premise'],
        'ask_for': wrong_info['ask-for'],
        'h1': wrong_info['hypothesis1'],
        'h2': wrong_info['hypothesis2'],
        'correct_answer': wrong_info['correct_answer'],
        'gemma_wrong_choice': wrong_info['gemma_choice'],
        'max_cluster_id': int(max_cluster_id),
        'max_cluster_size': int(np.sum(cluster_mask)),
        'n_clusters': len(cluster_sizes),
        'core_point_text': core_point_text,
        'topk_texts': topk_texts,
        'topk_representative_text': topk_representative_text,
        'core_point_vec': core_point_vec,
        'topk_vec': topk_vec,
        'centroid_vec': centroid
    }


# 处理所有错误题目
wrong_analysis_results = []

for idx in tqdm(wrong_indices, desc="分析错误题目"):
    if idx not in gemma_100_answers or len(gemma_100_answers[idx]) < 100:
        print(f"⚠️ 跳过 {idx}: 回答数不足 100")
        continue
    
    answers_list = gemma_100_answers[idx]
    wrong_info = wrong_results_dict[idx]
    
    result = analyze_wrong_question(idx, answers_list, wrong_info)
    wrong_analysis_results.append(result)

print(f"\n✅ 分析完成！共处理 {len(wrong_analysis_results)} 个错误题目")

# ========== 保存聚类分析结果到文件（用于内核重启后恢复） ==========
WRONG_ANALYSIS_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/wrong_analysis_results.jsonl"

with open(WRONG_ANALYSIS_FILE, 'w', encoding='utf-8') as f:
    for r in wrong_analysis_results:
        # 保存时排除向量数据（无法JSON序列化且后续步骤不需要）
        r_save = {k: v for k, v in r.items() if not k.endswith('_vec')}
        f.write(json.dumps(r_save, ensure_ascii=False) + "\n")

print(f"📁 聚类分析结果已保存到: {WRONG_ANALYSIS_FILE}")
print("   （切换环境重启内核后，可从 Cell 10 继续运行）")

对错误题目进行 HDBSCAN 聚类分析...


分析错误题目:   0%|          | 0/127 [00:00<?, ?it/s]/home/amax/miniconda3/envs/nvembed/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
分析错误题目:   1%|          | 1/127 [00:01<02:06,  1.00s/it]/home/amax/miniconda3/envs/nvembed/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
分析错误题目:   2%|▏         | 2/127 [00:01<01:12,  1.73it/s]/home/amax/miniconda3/envs/nvembed/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.


✅ 分析完成！共处理 127 个错误题目
📁 聚类分析结果已保存到: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/wrong_analysis_results.jsonl
   （切换环境重启内核后，可从 Cell 10 继续运行）


In [1]:
# Cell 10: 加载 Gemma 模型 (用于判断相似度)
# ========== 导入库 (内核重启后需要) ==========
import torch
import os
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Gemma 模型路径配置
GEMMA_CACHE_DIR = "/home/amax/Gemma_and_NVembed/Gemma"
GEMMA_MODEL_NAME = "google/gemma-2-9b-it"

# ========== 从文件恢复聚类分析结果（内核重启后使用） ==========
WRONG_ANALYSIS_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/wrong_analysis_results.jsonl"

if 'wrong_analysis_results' not in dir():
    print("📂 从文件恢复聚类分析结果...")
    wrong_analysis_results = []
    with open(WRONG_ANALYSIS_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                wrong_analysis_results.append(json.loads(line))
    print(f"✅ 已恢复 {len(wrong_analysis_results)} 个错误题目的分析结果")
else:
    print("✅ 聚类分析结果已在内存中")

print("\n" + "=" * 60)
print("加载 Gemma 模型...")
print("=" * 60)

# 查找本地模型路径
model_base = os.path.join(GEMMA_CACHE_DIR, "models--google--gemma-2-9b-it", "snapshots")
snapshots = os.listdir(model_base)
local_model_path = os.path.join(model_base, snapshots[0])
print(f"📁 本地模型路径: {local_model_path}")

# 加载 tokenizer
print("\n⏳ 正在加载 Gemma tokenizer...")
gemma_tokenizer = AutoTokenizer.from_pretrained(
    local_model_path,
    local_files_only=True
)
print("✅ Tokenizer 加载成功！")

# 加载模型
print("\n⏳ 正在加载 Gemma 模型（约需 1-2 分钟）...")
gemma_model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    local_files_only=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"✅ Gemma 模型加载成功！")
print(f"💻 模型设备: {gemma_model.device}")
print(f"📊 模型参数量: {gemma_model.num_parameters() / 1e9:.2f}B")

/home/amax/miniconda3/envs/gemma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 从文件恢复聚类分析结果...
✅ 已恢复 127 个错误题目的分析结果

加载 Gemma 模型...
📁 本地模型路径: /home/amax/Gemma_and_NVembed/Gemma/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819

⏳ 正在加载 Gemma tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Tokenizer 加载成功！

⏳ 正在加载 Gemma 模型（约需 1-2 分钟）...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.40it/s]

✅ Gemma 模型加载成功！
💻 模型设备: cuda:0
📊 模型参数量: 9.24B


In [6]:
from tqdm import tqdm
# Cell 11: 让 Gemma 判断 core_point/topk 回答与哪个选项更接近
print("=" * 60)
print("让 Gemma 判断代表性回答与哪个选项更接近...")
print("=" * 60)


def construct_similarity_prompt(representative_text, h1, h2, ask_for, premise):
    """
    构造让 Gemma 判断相似度的 prompt
    
    Args:
        representative_text: core_point 或 topk 对应的代表性回答
        h1: hypothesis1
        h2: hypothesis2
        ask_for: 'cause' 或 'effect'
        premise: 原始前提/背景
    
    Returns:
        prompt 字符串
    """
    if ask_for == 'cause':
        question_type = "What is the most likely cause of this situation?"
    else:
        question_type = "What is the most likely effect of this situation?"
    
    prompt = f"""You are a semantic similarity expert. Given a premise and a question, I will show you one generated answer and two candidate options. Please determine which option is more semantically similar to the generated answer.

Premise: {premise}
Question: {question_type}

Generated Answer: {representative_text}

Option A: {h1}
Option B: {h2}

Which option (A or B) is more semantically similar to the generated answer? Just output A or B, nothing else.

Answer:"""
    
    return prompt


def get_gemma_similarity_choice(representative_text, h1, h2, ask_for, premise):
    """
    让 Gemma 判断代表性回答与哪个选项更接近
    
    Returns:
        'A', 'B' 或 'unknown'
    """
    prompt = construct_similarity_prompt(representative_text, h1, h2, ask_for, premise)
    
    chat = [{"role": "user", "content": prompt}]
    prompt_formatted = gemma_tokenizer.apply_chat_template(
        chat, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = gemma_tokenizer(prompt_formatted, return_tensors="pt").to(gemma_model.device)
    
    with torch.no_grad():
        outputs = gemma_model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=gemma_tokenizer.eos_token_id
        )
    
    generated_text = gemma_tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:], 
        skip_special_tokens=True
    ).strip().upper()
    
    if 'A' in generated_text[:5]:
        return 'A', generated_text
    elif 'B' in generated_text[:5]:
        return 'B', generated_text
    else:
        return 'unknown', generated_text


# 对每个错误题目，用 core_point 和 topk 让 Gemma 重新判断
verification_results = []

for analysis in tqdm(wrong_analysis_results, desc="Gemma 重新判断"):
    idx = analysis['index']
    h1 = analysis['h1']
    h2 = analysis['h2']
    ask_for = analysis['ask_for']
    correct_answer = analysis['correct_answer']
    premise = analysis['premise']
    
    # 使用 core_point_text 让 Gemma 判断
    core_choice, core_raw = get_gemma_similarity_choice(
        analysis['core_point_text'], h1, h2, ask_for, premise
    )
    core_correct = (core_choice == correct_answer)
    
    # 使用 topk_representative_text 让 Gemma 判断
    topk_choice, topk_raw = get_gemma_similarity_choice(
        analysis['topk_representative_text'], h1, h2, ask_for, premise
    )
    topk_correct = (topk_choice == correct_answer)
    
    verification_results.append({
        'index': idx,
        'premise': analysis['premise'],
        'ask_for': ask_for,
        'h1': h1,
        'h2': h2,
        'correct_answer': correct_answer,
        'original_wrong_choice': analysis['gemma_wrong_choice'],
        'max_cluster_size': analysis['max_cluster_size'],
        'n_clusters': analysis['n_clusters'],
        # Core point 结果
        'core_point_text': analysis['core_point_text'],
        'core_point_choice': core_choice,
        'core_point_raw_answer': core_raw,
        'core_point_correct': core_correct,
        # TopK 结果
        'topk_representative_text': analysis['topk_representative_text'],
        'topk_choice': topk_choice,
        'topk_raw_answer': topk_raw,
        'topk_correct': topk_correct,
    })

print(f"\n✅ 重新判断完成！共处理 {len(verification_results)} 个题目")

让 Gemma 判断代表性回答与哪个选项更接近...


Gemma 重新判断: 100%|██████████| 127/127 [00:31<00:00,  4.08it/s]


✅ 重新判断完成！共处理 127 个题目


In [10]:
# Cell 12: 统计和分析结果
# 配置文件路径
OUTPUT_FILE = "/home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results.jsonl"

print("=" * 60)
print("📊 错误题目重新判断结果统计")
print("=" * 60)

# 总体统计
total_wrong = len(verification_results)
core_correct_count = sum(1 for r in verification_results if r['core_point_correct'])
topk_correct_count = sum(1 for r in verification_results if r['topk_correct'])
core_unknown_count = sum(1 for r in verification_results if r['core_point_choice'] == 'unknown')
topk_unknown_count = sum(1 for r in verification_results if r['topk_choice'] == 'unknown')

print(f"\n📈 总体统计 (原始错误题目数: {total_wrong}):")
print(f"   Core Point 方法:")
print(f"     - 修正正确: {core_correct_count}/{total_wrong} = {core_correct_count/total_wrong*100:.2f}%")
print(f"     - 无效回答: {core_unknown_count}")
print(f"   TopK 方法:")
print(f"     - 修正正确: {topk_correct_count}/{total_wrong} = {topk_correct_count/total_wrong*100:.2f}%")
print(f"     - 无效回答: {topk_unknown_count}")

# 按 ask-for 分类统计
cause_results = [r for r in verification_results if r['ask_for'] == 'cause']
effect_results = [r for r in verification_results if r['ask_for'] == 'effect']

print(f"\n📈 按 Ask-for 分类统计:")
print(f"\n   CAUSE 类 ({len(cause_results)} 个):")
if cause_results:
    cause_core_correct = sum(1 for r in cause_results if r['core_point_correct'])
    cause_topk_correct = sum(1 for r in cause_results if r['topk_correct'])
    print(f"     - Core Point 修正正确: {cause_core_correct}/{len(cause_results)} = {cause_core_correct/len(cause_results)*100:.2f}%")
    print(f"     - TopK 修正正确: {cause_topk_correct}/{len(cause_results)} = {cause_topk_correct/len(cause_results)*100:.2f}%")

print(f"\n   EFFECT 类 ({len(effect_results)} 个):")
if effect_results:
    effect_core_correct = sum(1 for r in effect_results if r['core_point_correct'])
    effect_topk_correct = sum(1 for r in effect_results if r['topk_correct'])
    print(f"     - Core Point 修正正确: {effect_core_correct}/{len(effect_results)} = {effect_core_correct/len(effect_results)*100:.2f}%")
    print(f"     - TopK 修正正确: {effect_topk_correct}/{len(effect_results)} = {effect_topk_correct/len(effect_results)*100:.2f}%")

# 保存结果
verification_output_file = OUTPUT_FILE.replace('.jsonl', '_verification_results.jsonl')
with open(verification_output_file, 'w', encoding='utf-8') as f:
    for r in verification_results:
        # 移除向量数据（太大了）
        r_save = {k: v for k, v in r.items() if not k.endswith('_vec')}
        f.write(json.dumps(r_save, ensure_ascii=False) + "\n")
print(f"\n📁 详细结果已保存到: {verification_output_file}")

# 保存统计结果
verification_stats = {
    'total_wrong': total_wrong,
    'core_point': {
        'correct': core_correct_count,
        'accuracy': core_correct_count / total_wrong * 100 if total_wrong > 0 else 0,
        'unknown': core_unknown_count
    },
    'topk': {
        'correct': topk_correct_count,
        'accuracy': topk_correct_count / total_wrong * 100 if total_wrong > 0 else 0,
        'unknown': topk_unknown_count
    },
    'by_ask_for': {
        'cause': {
            'total': len(cause_results),
            'core_correct': sum(1 for r in cause_results if r['core_point_correct']) if cause_results else 0,
            'topk_correct': sum(1 for r in cause_results if r['topk_correct']) if cause_results else 0,
        },
        'effect': {
            'total': len(effect_results),
            'core_correct': sum(1 for r in effect_results if r['core_point_correct']) if effect_results else 0,
            'topk_correct': sum(1 for r in effect_results if r['topk_correct']) if effect_results else 0,
        }
    }
}

verification_stats_file = OUTPUT_FILE.replace('.jsonl', '_verification_stats.json')

with open(verification_stats_file, 'w', encoding='utf-8') as f:
    json.dump(verification_stats, f, indent=2, ensure_ascii=False)

print(f"📁 统计结果已保存到: {verification_stats_file}")

📊 错误题目重新判断结果统计

📈 总体统计 (原始错误题目数: 127):
   Core Point 方法:
     - 修正正确: 46/127 = 36.22%
     - 无效回答: 0
   TopK 方法:
     - 修正正确: 47/127 = 37.01%
     - 无效回答: 0

📈 按 Ask-for 分类统计:

   CAUSE 类 (67 个):
     - Core Point 修正正确: 26/67 = 38.81%
     - TopK 修正正确: 25/67 = 37.31%

   EFFECT 类 (60 个):
     - Core Point 修正正确: 20/60 = 33.33%
     - TopK 修正正确: 22/60 = 36.67%

📁 详细结果已保存到: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results_verification_results.jsonl
📁 统计结果已保存到: /home/amax/Zixing_Jia/Vector-HASH-tinkering-main/Gemma_anwer_causal/gemma_choice_results_verification_stats.json


In [13]:
# Cell 13: 展示示例结果
print("=" * 60)
print("📋 示例结果展示")
print("=" * 60)

# 展示被 Core Point 修正成功的例子
core_corrected = [r for r in verification_results if r['core_point_correct']]
print(f"\n✅ Core Point 修正成功的示例 (共 {len(core_corrected)} 个):")
print("-" * 60)
for i, r in enumerate(core_corrected[:3], 1):
    print(f"\n[{i}] Index: {r['index']}")
    print(f"    Premise: {r['premise']}")
    print(f"    Ask-for: {r['ask_for']}")
    print(f"    A (H1): {r['h1']}")
    print(f"    B (H2): {r['h2']}")
    print(f"    正确答案: {r['correct_answer']}")
    print(f"    原始错误选择: {r['original_wrong_choice']}")
    print(f"    Core Point 回答: {r['core_point_text']}")
    print(f"    Core Point 选择: {r['core_point_choice']} ✅")

# 展示被 TopK 修正成功的例子
topk_corrected = [r for r in verification_results if r['topk_correct']]
print(f"\n\n✅ TopK 修正成功的示例 (共 {len(topk_corrected)} 个):")
print("-" * 60)
for i, r in enumerate(topk_corrected[:3], 1):
    print(f"\n[{i}] Index: {r['index']}")
    print(f"    Premise: {r['premise']}")
    print(f"    A (H1): {r['h1']}")
    print(f"    B (H2): {r['h2']}")
    print(f"    正确答案: {r['correct_answer']}")
    print(f"    TopK 代表回答: {r['topk_representative_text']}")
    print(f"    TopK 选择: {r['topk_choice']} ✅")

# 展示仍然错误的例子
still_wrong = [r for r in verification_results if not r['core_point_correct'] and not r['topk_correct']]
print(f"\n\n❌ 仍然错误的示例 (共 {len(still_wrong)} 个):")
print("-" * 60)
for i, r in enumerate(still_wrong[:3], 1):
    print(f"\n[{i}] Index: {r['index']}")
    print(f"    Premise: {r['premise']}")
    print(f"    Ask-for: {r['ask_for']}")
    print(f"    A (H1): {r['h1']}")
    print(f"    B (H2): {r['h2']}")
    print(f"    正确答案: {r['correct_answer']}")
    print(f"    Core Point 回答: {r['core_point_text']}")
    print(f"    Core Point 选择: {r['core_point_choice']} ❌")
    print(f"    TopK 回答: {r['topk_representative_text']}")
    print(f"    TopK 选择: {r['topk_choice']} ❌")

📋 示例结果展示

✅ Core Point 修正成功的示例 (共 46 个):
------------------------------------------------------------

[1] Index: train-1056
    Premise: Jack and Tom are relatives.
    Ask-for: effect
    A (H1): They live under the same roof.
    B (H2): They grew up.
    正确答案: A
    原始错误选择: B
    Core Point 回答: They might share a common ancestor.
    Core Point 选择: A ✅

[2] Index: train-3554
    Premise: The azaleas have grown in different ways.
    Ask-for: cause
    A (H1): Scientists have done a research about the azaleas with root rot.
    B (H2): Farmers weeded the land. .
    正确答案: A
    原始错误选择: B
    Core Point 回答: The azaleas received varying amounts of sunlight or water.
    Core Point 选择: A ✅

[3] Index: train-3594
    Premise: His conclusions are not based on new scientific evidence.
    Ask-for: effect
    A (H1): Others don't believe his conlusions.
    B (H2): His predictions were not accurate.
    正确答案: A
    原始错误选择: B
    Core Point 回答: His conclusions will likely be met with skepti